In [1]:
import pandas as pd 
import csv
import time
import requests
#import re
import openpyxl
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os

# 경로에 추가
os.environ["PATH"] += os.pathsep + r'C:\Users\YLee\Downloads\chromedriver_win32';

# 객체명을 적어야 한다. 'r'이나 ||를 적어줘야 encoding이 맞는다.
driver = webdriver.Chrome('C:\\Users\\YLee\\Downloads\\chromedriver_win32\\chromedriver.exe')
driver.implicitly_wait(3)
driver.get('https://klue.kr/')

# 로그인 버튼
driver.find_element_by_xpath('/html/body/app-root/menubar/div/ul/menubar-guest/span[2]').click()

#아이디, 비밀번호 입력하기
driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[1]').send_keys('eleph1021')
driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[2]').send_keys('4683841ice!')

#로그인 버튼을 눌러서 확인
driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/button').click()

#Alert Handling
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
try:
    WebDriverWait(driver, 3).until(EC.alert_is_present(),
                                   'Timed out waiting for PA creation ' +
                                   'confirmation popup to appear.')
    print("alert present")
    alert = driver.switch_to.alert()
    alert.accept()
    driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[1]').send_keys('eleph1021')
    driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[2]').send_keys('4683841ice!')
    driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/button').click()
except TimeoutException:
    print("no alert")


#강의 URL 접근 '생물심리학'
driver.get('https://klue.kr/lecture/91767')

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# scroll to a page with infinite loading
SCROLL_PAUSE_TIME = 1.5

    # Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

def score_rating(self):
    if 'width: 26px' in self: 
        return 1
    elif 'width: 52px' in self: 
        return 2
    elif 'width: 78px' in self: 
        return 3
    elif 'width: 104px' in self: 
        return 4
    elif 'width: 130px' in self:
        return 5
    

npo_reviews = {}   # create dict
review_no = 0 

reviews = soup.find_all('div', {'class':'lecture-eval-content'})

for review in reviews:
    lec_title = soup.find('div', {'class':'lecture-index-main-title'}).text
    lec_prof = soup.find('div', {'class':'lecture-title-professor'}).text[:-4]
    
    tds = soup.find_all('td')
    lec_semester = tds[0].get_text()[:-2]
    lec_code = tds[1].get_text()
    lec_type = tds[2].get_text()
    lec_time1 = tds[3].get_text() 
    lec_time2 = tds[5].get_text()
    lec_time = lec_time1 + ',' + lec_time2
    lec_credit = tds[4].get_text()[0]
    
    texts = soup.find_all('text')
    lec_attendrate = texts[5].get_text()
    lec_creditrate = texts[6].get_text()
    lec_difficultyrate = texts[7].get_text()
    lec_workrate = texts[8].get_text()
    lec_achieverate = texts[9].get_text()
    lec_totalrate = soup.find('span', {'class':'lecture-main-rate-point'}).text
    
    oldID = review.find('span', {'class':'lecture-eval-content-user'}).text[:-7]
    ID = oldID.replace("\n", "")
    date = review.find('span', {'class':'lecutre-eval-content-modtime'}).text[4:]
    comment = review.find('div', {'class':'lecture-eval-content-context'}).text
    
    scores = review.find_all('div', {'class':'slider-klue-outer'})
    pre_totalrate = scores[0].get('style')
    rev_totalrate = score_rating(pre_totalrate)
    pre_attendrate = scores[1].get('style')
    rev_attendrate = score_rating(pre_attendrate)
    pre_difficultyrate = scores[2].get('style')
    rev_difficultyrate = score_rating(pre_difficultyrate)
    pre_workrate = scores[3].get('style')
    rev_workrate = score_rating(pre_workrate)
    pre_creditrate = scores[4].get('style')
    rev_creditrate = score_rating(pre_creditrate)
    pre_achieverate = scores[5].get('style')
    rev_achieverate = score_rating(pre_achieverate)
    
    review_no += 1 
    npo_reviews[review_no] = [lec_title, lec_prof, lec_semester, lec_code, lec_type, lec_time, lec_credit,
                              lec_attendrate, lec_creditrate, lec_difficultyrate, lec_workrate, lec_achieverate,
                              lec_totalrate, ID, date, comment, rev_totalrate, rev_attendrate, rev_difficultyrate,
                              rev_workrate, rev_creditrate, rev_achieverate]
print("Total Reviews:", review_no)

# Create the workbook and sheet for Excel
workbook = openpyxl.Workbook()
sheet = workbook.active

header = ['','Lecture', 'Professor', 'Semester', 'Code', 'Type',
          'TIme', 'Credit', 'Lec_Attendrate', 'Lec_Creditrate',
          'Lec_Difficultyrate','Lec_Workrate', 'Lec_Achieverate',
          'Lec_totalrate','ID', 'Date', 'Comment', 'Rev_Totalrate',
          'Rev_Attendrate','Rev_Difficultyrate', 'Rev_Workrate',
          'Rev_Creditrate','Rev_Achieverate']

sheet.append(header)

# openpyxl does things based on 1 instead of 0
row = 2
for key,values in npo_reviews.items():
    # Put the key in the first column for each key in the dictionary
    sheet.cell(row=row, column=1, value=key)
    column = 2
    for element in values:
        # Put the element in each adjacent column for each element in the tuple
        sheet.cell(row=row, column=column, value=element)
        column += 1
    row += 1

workbook.save(filename="klue.xlsx")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/app-root/menubar/div/ul/menubar-guest/span[2]"}
  (Session info: chrome=76.0.3809.100)


# LIST ver.

In [12]:
import pandas as pd 
import csv
import time
import requests
#import re
import openpyxl
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os

# 경로에 추가
os.environ["PATH"] += os.pathsep + r'C:\Users\YLee\Downloads\chromedriver_win32';

# 객체명을 적어야 한다. 'r'이나 ||를 적어줘야 encoding이 맞는다.
driver = webdriver.Chrome('C:\\Users\\YLee\\Downloads\\chromedriver_win32\\chromedriver.exe')
driver.implicitly_wait(3)
driver.get('https://klue.kr/')

# 로그인 버튼
driver.find_element_by_xpath('/html/body/app-root/menubar/div/ul/menubar-guest/span[2]').click()

#아이디, 비밀번호 입력하기
driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[1]').send_keys('eleph1021')
driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[2]').send_keys('4683841ice!')

#로그인 버튼을 눌러서 확인
driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/button').click()

#Alert Handling
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
try:
    WebDriverWait(driver, 3).until(EC.alert_is_present(),
                                   'Timed out waiting for PA creation ' +
                                   'confirmation popup to appear.')
    print("alert present")
    alert = driver.switch_to.alert()
    alert.accept()
    driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[1]').send_keys('eleph1021')
    driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[2]').send_keys('4683841ice!')
    driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/button').click()
except TimeoutException:
    print("no alert")


#강의 URL 접근 '생물심리학'
driver.get('https://klue.kr/lecture/91767')

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# scroll to a page with infinite loading
SCROLL_PAUSE_TIME = 1.5

    # Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

def score_rating(self):
    if 'width: 26px' in self: 
        return 1
    elif 'width: 52px' in self: 
        return 2
    elif 'width: 78px' in self: 
        return 3
    elif 'width: 104px' in self: 
        return 4
    elif 'width: 130px' in self:
        return 5
    
reviews_list = []
npo_reviews = []
review_no = 0
reviews = soup.find_all('div', {'class':'lecture-eval-content'})

for review in reviews:
    lec_title = soup.find('div', {'class':'lecture-index-main-title'}).text
    lec_prof = soup.find('div', {'class':'lecture-title-professor'}).text[:-4]
    
    tds = soup.find_all('td')
    lec_semester = tds[0].get_text()[:-2]
    lec_code = tds[1].get_text()
    lec_type = tds[2].get_text()
    lec_time1 = tds[3].get_text() 
    lec_time2 = tds[5].get_text()
    lec_time = lec_time1 + ',' + lec_time2
    lec_credit = tds[4].get_text()[0]
    
    texts = soup.find_all('text')
    lec_attendrate = texts[5].get_text()
    lec_creditrate = texts[6].get_text()
    lec_difficultyrate = texts[7].get_text()
    lec_workrate = texts[8].get_text()
    lec_achieverate = texts[9].get_text()
    lec_totalrate = soup.find('span', {'class':'lecture-main-rate-point'}).text
    
    oldID = review.find('span', {'class':'lecture-eval-content-user'}).text[:-7]
    ID = oldID.replace("\n", "")
    date = review.find('span', {'class':'lecutre-eval-content-modtime'}).text[4:]
    comment = review.find('div', {'class':'lecture-eval-content-context'}).text
    
    scores = review.find_all('div', {'class':'slider-klue-outer'})
    pre_totalrate = scores[0].get('style')
    rev_totalrate = score_rating(pre_totalrate)
    pre_attendrate = scores[1].get('style')
    rev_attendrate = score_rating(pre_attendrate)
    pre_difficultyrate = scores[2].get('style')
    rev_difficultyrate = score_rating(pre_difficultyrate)
    pre_workrate = scores[3].get('style')
    rev_workrate = score_rating(pre_workrate)
    pre_creditrate = scores[4].get('style')
    rev_creditrate = score_rating(pre_creditrate)
    pre_achieverate = scores[5].get('style')
    rev_achieverate = score_rating(pre_achieverate)
    
    review_no += 1
    npo_reviews = [lec_title, lec_prof, lec_semester, lec_code, lec_type, lec_time, lec_credit,lec_attendrate, lec_creditrate, 
                   lec_difficultyrate, lec_workrate, lec_achieverate,lec_totalrate, ID, date, comment, rev_totalrate, 
                   rev_attendrate, rev_difficultyrate, rev_workrate, rev_creditrate, rev_achieverate]
    reviews_list.append(npo_reviews)
print("Total Reviews:", review_no)

no alert
Total Reviews: 34


In [13]:
# Create csv file  # 수정 바람
import csv 

with open('klue_list.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Lecture', 'Professor', 'Semester', 'Code', 'Type',
          'TIme', 'Credit', 'Lec_Attendrate', 'Lec_Creditrate',
          'Lec_Difficultyrate','Lec_Workrate', 'Lec_Achieverate',
          'Lec_totalrate','ID', 'Date', 'Comment', 'Rev_Totalrate',
          'Rev_Attendrate','Rev_Difficultyrate', 'Rev_Workrate',
          'Rev_Creditrate','Rev_Achieverate'])
    writer.writerow(reviews_list)

In [14]:
reviews_list

[['생물심리학(영강) ',
  '최준식',
  '2019-1',
  'PSYC271',
  '전공선택',
  ' 월(2) 법학관구관 B102호, 수(2) 법학관구관 B102호',
  '3',
  '3.3',
  '2.6',
  '4.2',
  '4.7',
  '3.9',
  '3.24',
  '보라나나       ',
  '2019-08-06 03:51:49',
  '수업은 재밌고 양 뇌 해부도 재밌고 어쩐지 굉장히 많이 배워가는 느낌으로 수업을 마치긴 했습니다만, 많이 배워가는 느낌이라는 건 달리 말하면 학습량이 좀 많다는 의미입니다. 특히 과목 자체가 "생물"심리학이니까요. 1학기때는 뇌 부위 외우기가 주관식으로 나오고 2학기때는 좀 랜덤해지는데, 저는 사실 뇌 부위 외우기가 조금 많이 힘들었습니다 ㅠ 영어로 써야 하는데 진짜 한 번도 그런 표현들을 들어본 적이 없었거든요. 생물 지식이 좀 있으시면 편하시지 않을까 합니다. 중간에 레포트 하나 있고 테드 강연 듣고 OX 퀴즈 맞추기 같은 게 있었는데, 테드는 텍스트 말고 영상을 잘 보시는 게 좋을 것 같아요. 레포트는 자유양식입니다만 교수님이 추후에 가이드라인을 주셨습니다. 저는...가이드라인 받기 전에 쓰고 힘들다고 그냥 내서 성적을 조금 버렸지만...참고해 가면서 하시면 잘 받으실 수 있을 것 같아요.출석은 매번 부르시긴 하지만 랜덤으로 소수 인원을 뽑아서 부르는 방식입니다.',
  4,
  4,
  4,
  4,
  2,
  5],
 ['생물심리학(영강) ',
  '최준식',
  '2019-1',
  'PSYC271',
  '전공선택',
  ' 월(2) 법학관구관 B102호, 수(2) 법학관구관 B102호',
  '3',
  '3.3',
  '2.6',
  '4.2',
  '4.7',
  '3.9',
  '3.24',
  '오9오9       ',
  '2019-08-06 02:25:19',
  '교수님이 열정이 넘치심. 실험도 하고 괜춘해음.',
  3,
  2,
  3,
  4,
  3,
  3]

In [ ]:


npo_reviews_df.to_csv('npo_reviews.csv')

In [124]:
import pandas as pd 
import csv
import time
import requests
import re
from bs4 import BeautifulSoup


# 경로에 추가
import os;
os.environ["PATH"] += os.pathsep + r'C:\Users\YLee\Downloads\chromedriver_win32';

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# 객체명을 적어야 한다. 'r'이나 ||를 적어줘야 encoding이 맞는다.
driver = webdriver.Chrome('C:\\Users\\YLee\\Downloads\\chromedriver_win32\\chromedriver.exe')
driver.implicitly_wait(3)
driver.get('https://klue.kr/')

# 로그인 버튼
driver.find_element_by_xpath('/html/body/app-root/menubar/div/ul/menubar-guest/span[2]').click()

#아이디, 비밀번호 입력하기
driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[1]').send_keys('eleph1021')
driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/input[2]').send_keys('4683841ice!')

#로그인 버튼을 눌러서 확인
driver.find_element_by_xpath('/html/body/app-root/app-modal/div/div/div/modal-contents/div/modal-login/button').click()

In [125]:
#강의 URL 접근 '생물심리학'
driver.get('https://klue.kr/lecture/91767')

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# scroll to a page with infinite loading
SCROLL_PAUSE_TIME = 1.5

    # Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    
# Scroll bottom to top
#driver.execute_script("window.scrollTo(document.body.scrollHeight, 0);")

In [159]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

def score_rating(self):
    if 'width: 26px' in self: 
        return 1
    elif 'width: 52px' in self: 
        return 2
    elif 'width: 78px' in self: 
        return 3
    elif 'width: 104px' in self: 
        return 4
    elif 'width: 130px' in self:
        return 5
    

npo_reviews = {}   # create dict
review_no = 0 

reviews = soup.find_all('div', {'class':'lecture-eval-content'})

for review in reviews:
    lec_title = soup.find('div', {'class':'lecture-index-main-title'}).text
    lec_prof = soup.find('div', {'class':'lecture-title-professor'}).text[:-4]
    
    tds = soup.find_all('td')
    lec_semester = tds[0].get_text()[:-2]
    lec_code = tds[1].get_text()
    lec_type = tds[2].get_text()
    lec_time1 = tds[3].get_text() 
    lec_time2 = tds[5].get_text()
    lec_time = lec_time1 + ',' + lec_time2
    lec_credit = tds[4].get_text()[0]
    
    texts = soup.find_all('text')
    lec_attendrate = texts[5].get_text()
    lec_creditrate = texts[6].get_text()
    lec_difficultyrate = texts[7].get_text()
    lec_workrate = texts[8].get_text()
    lec_achieverate = texts[9].get_text()
    lec_totalrate = soup.find('span', {'class':'lecture-main-rate-point'}).text
    
    oldID = review.find('span', {'class':'lecture-eval-content-user'}).text[:-7]
    ID = oldID.replace("\n", "")
    date = review.find('span', {'class':'lecutre-eval-content-modtime'}).text[4:]
    comment = review.find('div', {'class':'lecture-eval-content-context'}).text
    
    scores = review.find_all('div', {'class':'slider-klue-outer'})
    pre_totalrate = scores[0].get('style')
    rev_totalrate = score_rating(pre_totalrate)
    pre_attendrate = scores[1].get('style')
    rev_attendrate = score_rating(pre_attendrate)
    pre_difficultyrate = scores[2].get('style')
    rev_difficultyrate = score_rating(pre_difficultyrate)
    pre_workrate = scores[3].get('style')
    rev_workrate = score_rating(pre_workrate)
    pre_creditrate = scores[4].get('style')
    rev_creditrate = score_rating(pre_creditrate)
    pre_achieverate = scores[5].get('style')
    rev_achieverate = score_rating(pre_achieverate)
    
    review_no += 1 
    npo_reviews[review_no] = [lec_title, lec_prof, lec_semester, lec_code, lec_type, lec_time, lec_credit,
                              lec_attendrate, lec_creditrate, lec_difficultyrate, lec_workrate, lec_achieverate,
                              lec_totalrate, ID, date, comment, rev_totalrate, rev_attendrate, rev_difficultyrate,
                              rev_workrate, rev_creditrate, rev_achieverate]
print("Total Reviews:", review_no)

Total Reviews: 34


In [171]:
import openpyxl

# Create the workbook and sheet for Excel
workbook = openpyxl.Workbook()
sheet = workbook.active

header = ['','Lecture', 'Professor', 'Semester', 'Code', 'Type',
          'TIme', 'Credit', 'Lec_Attendrate', 'Lec_Creditrate',
          'Lec_Difficultyrate','Lec_Workrate', 'Lec_Achieverate',
          'Lec_totalrate','ID', 'Date', 'Comment', 'Rev_Totalrate',
          'Rev_Attendrate','Rev_Difficultyrate', 'Rev_Workrate',
          'Rev_Creditrate','Rev_Achieverate']

sheet.append(header)

# openpyxl does things based on 1 instead of 0
row = 2
for key,values in npo_reviews.items():
    # Put the key in the first column for each key in the dictionary
    sheet.cell(row=row, column=1, value=key)
    column = 2
    for element in values:
        # Put the element in each adjacent column for each element in the tuple
        sheet.cell(row=row, column=column, value=element)
        column += 1
    row += 1

workbook.save(filename="klue.xlsx")

# 연습장

In [6]:
reviews_df = pd.DataFrame.from_list(npo_reviews, orient = 'index', columns = ['Lecture', 'Professor', 
                                                                                 'Semester', 'Code', 
                                                                                 'Type', 'TIme', 'Credit', 'Lec_Attendrate', 
                                                                                 'Lec_Creditrate','Lec_Difficultyrate', 
                                                                                 'Lec_Workrate', 'Lec_Achieverate','Lec_totalrate', 
                                                                                 'ID', 'Date', 'Comment', 'Rev_Totalrate', 'Rev_Attendrate',
                                                                                 'Rev_Difficultyrate', 'Rev_Workrate', 'Rev_Creditrate',
                                                                                 'Rev_Achieverate'])
reviews_df

AttributeError: type object 'DataFrame' has no attribute 'from_list'

In [157]:
npo_reviews_df.to_csv('npo_reviews.csv', encoding='utf-8', index=False)
df_csv = pd.read_csv('npo_reviews.csv')
df_csv

,Lecture,Professor,Semester,Code,Type,TIme,Credit,Lec_Attendrate,Lec_Creditrate,Lec_Difficultyrate,...,Lec_totalrate,ID,Date,Comment,Rev_Totalrate,Rev_Attendrate,Rev_Difficultyrate,Rev_Workrate,Rev_Creditrate,Rev_Achieverate
0,생물심리학(영강),최준식,2019-1,PSYC271,전공선택,"월(2) 법학관구관 B102호, 수(2) 법학관구관 B102호",3,3.3,2.6,4.2,...,3.24,\n보라나나\n,2019-08-06 03:51:49,수업은 재밌고 양 뇌 해부도 재밌고 어쩐지 굉장히 많이 배워가는 느낌으로 수업을 마...,4,4,4,4,2,5
1,생물심리학(영강),최준식,2019-1,PSYC271,전공선택,"월(2) 법학관구관 B102호, 수(2) 법학관구관 B102호",3,3.3,2.6,4.2,...,3.24,\n오9오9\n,2019-08-06 02:25:19,교수님이 열정이 넘치심. 실험도 하고 괜춘해음.,3,2,3,4,3,3
2,생물심리학(영강),최준식,2019-1,PSYC271,전공선택,"월(2) 법학관구관 B102호, 수(2) 법학관구관 B102호",3,3.3,2.6,4.2,...,3.24,\n땅강아지\n,2019-08-05 21:49:57,출석+색칠과제(완성만 하면 만점)+퀴즈(TED 영상 관련)+중간/기말고사+기말 레포...,3,3,4,4,2,3
3,생물심리학(영강),최준식,2019-1,PSYC271,전공선택,"월(2) 법학관구관 B102호, 수(2) 법학관구관 B102호",3,3.3,2.6,4.2,...,3.24,\n그레이스b\n,2019-08-05 17:23:18,전반적으로 학습량도 꽤 많은 편이고 영강인데다 내용도 쉽지 않아서 시간 투자를 많이...,4,3,4,5,3,5
4,생물심리학(영강),최준식,2019-1,PSYC271,전공선택,"월(2) 법학관구관 B102호, 수(2) 법학관구관 B102호",3,3.3,2.6,4.2,...,3.24,\n따란\n,2019-08-03 20:16:48,"평가방법: 짧은 자기소개 레포트, 중간, 기말, 뇌색칠과제, 퀴즈2번, 5페이지 레...",4,4,5,5,3,5
5,생물심리학(영강),최준식,2019-1,PSYC271,전공선택,"월(2) 법학관구관 B102호, 수(2) 법학관구관 B102호",3,3.3,2.6,4.2,...,3.24,\ncowsandmoos\n,2019-08-02 01:09:23,중간기말 두번보고 테드영상보고 쉽게 풀수있는 퀴즈 두번출첵은 랜덤으로 돌리심내용 오...,4,3,4,5,4,5
6,생물심리학(영강),최준식,2019-1,PSYC271,전공선택,"월(2) 법학관구관 B102호, 수(2) 법학관구관 B102호",3,3.3,2.6,4.2,...,3.24,\nteeeheee\n,2019-08-01 02:19:56,열심히 하면 정말 많은 것을 배우게 되는 좋은 수업이지만 학점은 진짜 안 주세요ㅠㅠ...,4,4,3,5,2,5
7,생물심리학(영강),최준식,2019-1,PSYC271,전공선택,"월(2) 법학관구관 B102호, 수(2) 법학관구관 B102호",3,3.3,2.6,4.2,...,3.24,\n게살수프\n,2019-08-01 01:48:32,그다지 어려운 내용이 아님에도 불구하고 일일히 암기해야 하기 때문에 중간 기말의 난...,4,2,4,5,3,3
8,생물심리학(영강),최준식,2019-1,PSYC271,전공선택,"월(2) 법학관구관 B102호, 수(2) 법학관구관 B102호",3,3.3,2.6,4.2,...,3.24,\n호르몽\n,2019-08-01 01:44:02,나는 생공에 뇌인지 융합과학이라 거의 모든 것들을 아는 상태로 들었다. 물론 시험은...,3,2,1,4,2,4
9,생물심리학(영강),최준식,2019-1,PSYC271,전공선택,"월(2) 법학관구관 B102호, 수(2) 법학관구관 B102호",3,3.3,2.6,4.2,...,3.24,\n뀨뀨239\n,2019-07-30 23:15:35,출석체크는 랜덤입니다. 심리학과 수업 중 대형강의임에도 출석체크를 하는 강의이기 때...,1,3,5,4,2,4


In [158]:
npo_reviews

{1: ['생물심리학(영강) ',
  '최준식',
  '2019-1',
  'PSYC271',
  '전공선택',
  ' 월(2) 법학관구관 B102호, 수(2) 법학관구관 B102호',
  '3',
  '3.3',
  '2.6',
  '4.2',
  '4.7',
  '3.9',
  '3.24',
  '\n보라나나\n       ',
  '2019-08-06 03:51:49',
  '수업은 재밌고 양 뇌 해부도 재밌고 어쩐지 굉장히 많이 배워가는 느낌으로 수업을 마치긴 했습니다만, 많이 배워가는 느낌이라는 건 달리 말하면 학습량이 좀 많다는 의미입니다. 특히 과목 자체가 "생물"심리학이니까요. 1학기때는 뇌 부위 외우기가 주관식으로 나오고 2학기때는 좀 랜덤해지는데, 저는 사실 뇌 부위 외우기가 조금 많이 힘들었습니다 ㅠ 영어로 써야 하는데 진짜 한 번도 그런 표현들을 들어본 적이 없었거든요. 생물 지식이 좀 있으시면 편하시지 않을까 합니다. 중간에 레포트 하나 있고 테드 강연 듣고 OX 퀴즈 맞추기 같은 게 있었는데, 테드는 텍스트 말고 영상을 잘 보시는 게 좋을 것 같아요. 레포트는 자유양식입니다만 교수님이 추후에 가이드라인을 주셨습니다. 저는...가이드라인 받기 전에 쓰고 힘들다고 그냥 내서 성적을 조금 버렸지만...참고해 가면서 하시면 잘 받으실 수 있을 것 같아요.출석은 매번 부르시긴 하지만 랜덤으로 소수 인원을 뽑아서 부르는 방식입니다.',
  4,
  4,
  4,
  4,
  2,
  5],
 2: ['생물심리학(영강) ',
  '최준식',
  '2019-1',
  'PSYC271',
  '전공선택',
  ' 월(2) 법학관구관 B102호, 수(2) 법학관구관 B102호',
  '3',
  '3.3',
  '2.6',
  '4.2',
  '4.7',
  '3.9',
  '3.24',
  '\n오9오9\n       ',
  '2019-08-06 02:25:19',
  '교수님이 열정이 넘치심. 실험도 하고 괜춘해음.',
  3,
  2,
  3,


In [ ]:
import csv
from selenium import webdriver
import selenium.webdriver.common.keys

with open('klue.csv','w', encoding='utf-8', newline='') as f:
    csv_columns = ['Lecture', 'Professor', 'Semester', 'Code', 'Type', 
               'TIme', 'Credit', 'Lec_Attendrate', 'Lec_Creditrate',
               'Lec_Difficultyrate','Lec_Workrate', 'Lec_Achieverate',
               'Lec_totalrate','ID', 'Date', 'Comment', 'Rev_Totalrate',
               'Rev_Attendrate','Rev_Difficultyrate', 'Rev_Workrate',
               'Rev_Creditrate','Rev_Achieverate']
    writer = csv.DictWriter(f, fieldnames=csv_columns)
    writer.writeheader()
    for data in npo_reviews:
        writer.writerow(data)
print("writing completed")